In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import gym
from gym_ianna.envs.ianna_env import IANNAEnv
import tensorflow as tf
import scipy as sp


In [3]:
sp.__version__, sp.__file__

('0.19.0',
 '/home/ihadanny/anaconda2/envs/py3k/lib/python3.6/site-packages/scipy/__init__.py')

In [4]:
import gym_ianna.envs.ianna_env as ianna_env

ianna_env.__file__

'/home/ihadanny/src/IANNA/gym_ianna/envs/ianna_env.py'

In [5]:
env_d = 'IANNA-v0'
#env_d = 'CartPole-v0'

#env params
if env_d == 'IANNA-v0':
    START_STATE_FROM = 0 # last 15 bits of the IANNA state are the groupby bits
    OP_NUMBER = 15          # how many fields can we turn on
    STATE_INPUT_SIZE = 51   # how many fields can we observe 
    MAX_STEPS = 7          # how many steps must we play in each episode

if env_d == 'CartPole-v0':
    OP_NUMBER = 2
    STATE_INPUT_SIZE = 4
    MAX_STEPS = 200

#nn params
HIDDEN_SIZE = 30

#discount rewards params
GAMMA = 0.99

#ADAM Optimizer hyper-parameters:
LEARNING_RATE = 0.01
B1 = 0.8
B2= 0.999
EPSILON=1e-6

#learning params
TOTAL_EPISODES = 1000
BATCH_NUMBER = 20
DISPLAY_FREQ = 100


In [6]:
#Initializing 
tf.reset_default_graph()

W1 = tf.get_variable(shape=[HIDDEN_SIZE,STATE_INPUT_SIZE],name='w1',
                      initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(shape=[HIDDEN_SIZE,HIDDEN_SIZE],name='w2',
                      initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable(shape=[OP_NUMBER,HIDDEN_SIZE],name='w3',
                      initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.get_variable(shape=[HIDDEN_SIZE,1],name='b1',
                      initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable(shape=[HIDDEN_SIZE,1],name='b2',
                      initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable(shape=[OP_NUMBER,1],name='b3',
                      initializer=tf.contrib.layers.xavier_initializer())

#Layers:
x = tf.placeholder(tf.float32, shape=[STATE_INPUT_SIZE,None],name='x')
h1 = tf.tanh(tf.matmul(W1,x) + b1)
h2 = tf.tanh(tf.matmul(W2,h1) + b2)
y = tf.nn.softmax(tf.matmul(W3,h2) + b3,dim=0)


In [7]:
saver = tf.train.Saver()
sess = tf.InteractiveSession()



In [8]:
#Loss function:
curr_reward = tf.placeholder(shape=[None],dtype=tf.float32)
actions_array = tf.placeholder(shape=[None],dtype=tf.int32)
pai_array = tf.gather(y,actions_array)
L = -tf.reduce_mean(tf.log(pai_array)*curr_reward)
gradient_holders = []
gradients = tf.gradients(L,tf.trainable_variables())


/home/ihadanny/anaconda2/envs/py3k/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [9]:
tvars = tf.trainable_variables()
#Initialize gradient lists for each trainable variable:
for idx,var in enumerate(tvars):
    placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
    gradient_holders.append(placeholder)

In [10]:

#Update mechanism:
adam = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE,beta1=B1,beta2=B2,epsilon=EPSILON)
update_batch = adam.apply_gradients(zip(gradient_holders,tvars))


In [11]:
# grad buffer is initialized to all zeros. 
# It's used to accumulate the gradients and is a regular variable, NOT a tf variable
init = tf.global_variables_initializer()
sess.run(init)
#saver.restore(sess, "../models/ianna-nn-supervised")

grad_buffer = sess.run(tf.trainable_variables())

def reset_grad_buffer():
    for ix,grad in enumerate(grad_buffer):
        grad_buffer[ix] = grad * 0


In [12]:
def get_action(sess,observation):
    """
    Given an observation, return action sampled according to the probabilities of the NN output
    """
    a_dist = sess.run(y,feed_dict={x:np.reshape(observation,(STATE_INPUT_SIZE, 1))})
    a = np.random.choice(range(OP_NUMBER),p=a_dist.reshape((OP_NUMBER)))        
    return a


In [13]:
def train(sess,cur_states_array,cur_actions_array,cur_curr_reward):
    """
    NN training procedure: Given arrays of states(observations),
    actions and rewards it computes the derivatives of the loss function
    then add the derivation values to the buffer 
    """

    G = sess.run(gradients,feed_dict={x:cur_states_array,actions_array:cur_actions_array,curr_reward:cur_curr_reward})
    for idx,grad in enumerate(G):
        grad_buffer[idx] += grad


In [14]:
def update(sess):
    """
    NN update procedure: apply the gradients to the NN variables
    """
    feed_dict = dict(zip(gradient_holders, grad_buffer))
    _ = sess.run(update_batch, feed_dict=feed_dict)


In [15]:
#        IANNA actions would be:
#        0) action_type:            back[0], filter[1], group[2]
#        1) col_id:                 [0..num_of_columns-1]
#        2) filter_operator:        LT[0], GT[1] if the selected column was numeric (maybe change semantics if column is STR?)
#        3) filter_decile:          [0..9] the filter operand  
#        4) aggregation column_id:  [0..num_of_columns - 1] (what do we do if the selected col is also grouped_by?)
#        5) aggregation type:       MEAN[0], COUNT[1], SUM[2], MIN[3], MAX[4]

def build_ianna_action_from_grouped_by_field(grouped_by_field):
    action = [2, grouped_by_field, 0, 0, 0, 0]
    return action

In [16]:
def project_state_to_nn_input(x):
    return x[START_STATE_FROM:START_STATE_FROM+STATE_INPUT_SIZE]

In [17]:

def discount_rewards(arr):
    """
    Helper function for computing discounted rewards,
    then the delayed rewards are normalized by the mean and std as requested.
    """
    discounts = np.zeros_like(arr)
    reward = 0
    for i in reversed(range(arr.size)):
        reward=GAMMA*(arr[i]+reward)
        discounts[i] = reward
    # following 3 lines destroy everything when the game is really simple: 
    # pick 4 fields out of 5 without repeating yourself
    #mean = np.mean(discounts,keepdims=True)
    #discounts = discounts - mean
    #discounts = discounts/ np.std(discounts)
    return discounts


In [18]:

env = gym.make(env_d)
total_episodes=TOTAL_EPISODES
batch_number = BATCH_NUMBER



[2017-09-29 11:48:57,046] Making new env: IANNA-v0


reading input /home/ihadanny/src/IANNA/gym_ianna/envs/../../Data_manipulation/raw_datasets/1.tsv
(8648, 15)
by_all [  3.44156700e-02   3.44156700e-02   3.90794293e-02   9.88289516e-01
   8.50589800e-01   8.68269483e-02   5.40944101e-02   1.00000000e+00
   5.23395009e-01   4.81127236e-01   5.60831487e-01   7.59601617e-01
   2.31267345e-04   2.31267345e-04   3.46901018e-04   9.42298797e-01
   4.27001388e-01   2.09393799e-02   1.15633673e-03   1.00000000e+00
   1.36447734e-02   1.62357808e-01   2.15098242e-01   2.12215321e-01
   8.64800000e+03   8.64800000e+03   8.64800000e+03   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00]
by_n

In [19]:
episode_number = 0

rewards = []
steps=[]
max_reward=0

reset_grad_buffer()

In [20]:

while episode_number <= total_episodes:
    for ep in range(batch_number):
        obsrv = project_state_to_nn_input(env.reset())
        ep_history=[]
        step_num=0
        total_reward=0
        done=False

        while not done and step_num < MAX_STEPS:
            #Perform the game "step:"
            step_num+=1
            action = get_action(sess,obsrv)
            if env_d == 'IANNA-v0':
                complex_action = build_ianna_action_from_grouped_by_field(action)
                obsrv1, reward, done, info = env.step(complex_action)
            else:
                obsrv1, reward, done, info = env.step(action)

            total_reward+=reward
            ep_history.append((np.array(obsrv),action,reward))
            obsrv=project_state_to_nn_input(obsrv1)

        episode_number+=1
        ep_history= np.array(ep_history)   
        ep_history[:,2] = discount_rewards(ep_history[:,2])

        """
        perform the training step, 
        feeding the network with the ep_history that contains
        the states,actions, and discounted rewards
        """
        ep_states_array = np.vstack(ep_history[:,0]).T
        ep_actions_array = ep_history[:,1].T
        ep_curr_reward = ep_history[:,2].T
        L=train(sess, ep_states_array, ep_actions_array, ep_curr_reward)

        #update the rewards/steps counter, storing the data for all episodes
        rewards.append(total_reward)
        steps.append(step_num)    
        
        if episode_number%DISPLAY_FREQ==0:
            print("latest game actions: ", ep_actions_array.T)
            print("latest game reward: ", total_reward)
            print("latest game first state: ", ep_states_array.T[0])
            print("latest game last state: ", obsrv)
            print("Total episodes: %d"%episode_number)
            print("Average steps per %d episodes: %f"%(DISPLAY_FREQ, np.mean(steps[-DISPLAY_FREQ:])))
            print("Average reward per %d episodes : %f"%(DISPLAY_FREQ, np.mean(rewards[-DISPLAY_FREQ:])))
    update(sess)
    reset_grad_buffer()
    if np.mean(rewards[-batch_number:])>max_reward:
        max_reward=np.mean(rewards[-batch_number:])
        print("\t\t\tCurr Max mean reward per batch:",max_reward)
  
        

			Curr Max mean reward per batch: 2.3
			Curr Max mean reward per batch: 2.4
			Curr Max mean reward per batch: 2.45


KeyboardInterrupt: 

In [ ]:
obsrv

In [ ]:
#sess.close()

In [ ]:
obs